# Part I: Get a list of pubmed ids to add in the unlabeled set

In [1]:
import sys , os 
import urllib, pickle

In [2]:
from pubmed_utils import get_pubmed_citations,get_pubmed_similar_docs

In [3]:
# LOAD from pickle
with open('/home/antonis/data/biocreative6/pubmed_ids_validation.pickle', 'rb') as f:
    pubmed_ids = pickle.load(f)

In [4]:
#define from which ids to take citations/similar articles from
ids = pubmed_ids['validation']+pubmed_ids['train']
print 'Will search for similar documents/citations from %i PubMed articles'%len(ids)

Will search for similar documents/citations from 1632 PubMed articles


## Get ids of *similar abstracts*

In [5]:
pubmed_ids['similar25'] = get_pubmed_similar_docs(ids,top_n=25)

Getting 25 most similar docs to doc 0...
Getting 25 most similar docs to doc 100...
Getting 25 most similar docs to doc 200...
Getting 25 most similar docs to doc 300...
Getting 25 most similar docs to doc 400...
Getting 25 most similar docs to doc 500...
Getting 25 most similar docs to doc 600...
Getting 25 most similar docs to doc 700...
Getting 25 most similar docs to doc 800...
Getting 25 most similar docs to doc 900...
Getting 25 most similar docs to doc 1000...
Getting 25 most similar docs to doc 1100...
Getting 25 most similar docs to doc 1200...
Getting 25 most similar docs to doc 1300...
Getting 25 most similar docs to doc 1400...
Getting 25 most similar docs to doc 1500...
Getting 25 most similar docs to doc 1600...


In [6]:
# delete from citations documents contained on train,dev, test set

print '# similar docs retrieved:',len(pubmed_ids['similar25'])
pubmed_ids['similar25'] = list(set(pubmed_ids['similar25']) - set(pubmed_ids['validation'] + pubmed_ids['train'] + pubmed_ids['test_gs']))
print '# similar docs without overlap with train/val:',len(pubmed_ids['similar25'])


# similar docs retrieved: 38541
# similar docs without overlap with train/val: 38311


In [7]:
pubmed_ids.keys()

['incoming_citations',
 'test_gs',
 'outgoing_citations',
 'train',
 'similar25',
 'validation']

In [8]:
# save to pickle
with open('/home/antonis/data/biocreative6/pubmed_ids_similar25.pickle', 'wb') as f:
    pickle.dump(pubmed_ids, f)

## Get ids of *citations*

In [ ]:
pubmed_ids['outgoing_citations'] = get_pubmed_citations(ids,outgoing = True)
pubmed_ids['incoming_citations'] = get_pubmed_citations(ids,outgoing = False)


In [62]:
pubmed_ids.keys()

['development',
 'incoming_citations',
 'training',
 'test_gs',
 'outgoing_citations',
 'chemdner_silver']

### delete from citations documents contained on train,dev, test set

In [15]:
print '# outgoing_citations:',len(pubmed_ids['outgoing_citations'])
pubmed_ids['outgoing_citations'] = list(set(pubmed_ids['outgoing_citations']) - set(pubmed_ids['development'] + pubmed_ids['training'] + pubmed_ids['test_gs']))
print '# outgoing_citations without overlap with train/test/dev:',len(pubmed_ids['outgoing_citations'])

print '# incoming_citations:',len(pubmed_ids['incoming_citations'])
pubmed_ids['incoming_citations'] = list(set(pubmed_ids['incoming_citations']) - set(pubmed_ids['development'] + pubmed_ids['training'] + pubmed_ids['test_gs']))
print '# incoming_citations without overlap with train/test/dev:',len(pubmed_ids['incoming_citations'])


# outgoing_citations: 22973
# outgoing_citations without overlap with traintestdev: 22251
# incoming_citations: 29059
# incoming_citations without overlap with traintestdev: 27546


In [16]:
with open('/home/antonis/data/biocreative6/pubmed_ids_citations.pickle', 'wb') as f:
    pickle.dump(pubmed_ids, f)

In [64]:
# Check some of the articles

In [65]:
# import random
# # get article link
# print 'https://www.ncbi.nlm.nih.gov/pubmed/?term='+citations[random.randint(0,len(citations))]
# # get article entities
# print "https://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/RESTful/tmTool.cgi/Bioconcept/%s/PubTator/"%citations[random.randint(0,len(citations))]


# Part II: Parse selected abstract + their NER tags

In [9]:
import sys , os, urllib, random, time

In [10]:
# load from pickle
with open('/home/antonis/data/biocreative6/pubmed_ids_similar25.pickle', 'rb') as f:
    pubmed_ids = pickle.load(f)

In [11]:
pubmed_ids.keys()

['incoming_citations',
 'test_gs',
 'outgoing_citations',
 'train',
 'similar25',
 'validation']

In [12]:
split_type = 'similar25'
bioconcept = 'Bioconcept' # includes all formats (genes, chemicals, diseases...)
api_output_loc = 'temp/NCBI_downloaded.txt'

In [14]:
# Get only a subset for starters
docs2retrieve = pubmed_ids[split_type] # or get a sample of it
# docs2retrieve = [ pubmed_ids[split_type][i] for i in sorted(random.sample(xrange(len(pubmed_ids[split_type])), 10000)) ]


In [31]:
print 'Will parse %i docs from %s set'%(len(docs2retrieve),split_type)
format = 'PubTator'

Will parse 38311 docs from similar25 set


In [25]:
# set Pubtator converter options
out_format = 'standoff'
data_path = '/home/antonis/data/biocreative6/NCBI_parsed/'
out_dir = os.path.join(data_path, split_type)
print "Will save in ", out_dir

Will save in  /home/antonis/data/biocreative6/NCBI_parsed/similar25


In [17]:
# import pubtator conversion script
sys.path.append('/home/antonis/ipythonNotebooks/snorkel-ml/helpers/pubtator/')
import convertpubtator

class arg_options:
    '''
    Class to overwrite pubtators tool main argument parser.
    '''
    def __init__(self, files, out_dir = out_dir , out_format='standoff', ids = None, ):
        self.ids = ids
        self.output = out_dir
        self.random = None
        self.segment = False
        self.files = files
        self.format = out_format
        self.no_text = False
        self.subdirs = False

# set conversion options
options = arg_options(files = api_output_loc, ids = None, out_format= 'standoff')

In [18]:
# Start downloading

In [33]:
n=199 #split to chunks
st_time = time.time()

for i in xrange(0,len(docs2retrieve), n):
    
    print 'Parsing docs %i to %i  . . . .'%(i+1,i+n+1)
    
    pmids = ','.join(docs2retrieve[i:i+n])
    
    all_pmids
#     print pmids
    
    # Calculate how much time to wait (avoid reaching pubmed API limits ~ 3 requests/sec )
    time_elapsed = time.time() - st_time
    if time_elapsed<0.34:
        time.sleep(0.34 - time_elapsed)
    
    url_Submit = "https://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/RESTful/tmTool.cgi/%s/%s/%s/"%(bioconcept,pmids,format)
    urllib.urlretrieve(url_Submit, api_output_loc)
    
    st_time = time.time() #refresh time

    print 'Converting from PubTator to BRAT'
    convertpubtator.convert(api_output_loc, convertpubtator.write_standoff, options = options)

Parsing docs 1 to 200  . . . .
Converting from PubTator to BRAT


Processed 100 documents ...
Done, processed 199 documents.


Parsing docs 200 to 399  . . . .
Converting from PubTator to BRAT


Processed 100 documents ...
Done, processed 199 documents.


Parsing docs 399 to 598  . . . .
Converting from PubTator to BRAT


...

Processed 100 documents ...
Done, processed 199 documents.


Parsing docs 38010 to 38209  . . . .
Converting from PubTator to BRAT


Processed 100 documents ...
Done, processed 199 documents.


Parsing docs 38209 to 38408  . . . .
Converting from PubTator to BRAT


Processed 100 documents ...
Done, processed 103 documents.
